In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
from sklearn.datasets import make_classification

class LogisticRegression:
    def __init__(self, learning_rate=0.01, n_epochs=50):
        self.lr = learning_rate
        self.n_epochs = n_epochs
        self.weights = None
        self.bias = None
        self.loss_history = []
        self.frames = []  # Store plots as frames in memory
    
    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Stability trick
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    def fit(self, X, y, save_path="Multiclass_LogisticRegression.gif"):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))  # Number of classes
        
        # One-hot encoding the labels
        y_one_hot = np.eye(n_classes)[y]
        
        self.weights = np.zeros((n_features, n_classes)) + 1
        self.bias = np.zeros(n_classes) + 10
    
        for epoch in range(self.n_epochs):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self.softmax(linear_model)

            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y_one_hot))
            db = (1 / n_samples) * np.sum(y_pred - y_one_hot, axis=0)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

            # Compute categorical cross-entropy loss
            loss = -np.mean(np.sum(y_one_hot * np.log(y_pred + 1e-9), axis=1))
            self.loss_history.append(loss)

            # Save plot
            self._save_plot(X, y, epoch, n_classes)
        
        # Save GIF after training
        imageio.mimsave(save_path, self.frames, duration=0.1)
    
    def _save_plot(self, X, y, epoch, n_classes):
        fig, ax = plt.subplots(1, 2, figsize=(12, 5))
        
        # Decision boundary
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                             np.linspace(y_min, y_max, 100))
        
        Z = np.dot(np.c_[xx.ravel(), yy.ravel()], self.weights) + self.bias
        Z = np.argmax(self.softmax(Z), axis=1)
        Z = Z.reshape(xx.shape)
        
        ax[0].contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
        scatter = ax[0].scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', edgecolors='k')
        legend1 = ax[0].legend(*scatter.legend_elements(), title="Classes")
        ax[0].add_artist(legend1)

        ax[0].set_title(f'Decision Boundary - Epoch {epoch+1}')
        ax[0].set_xlabel('Feature 1')
        ax[0].set_ylabel('Feature 2')
        ax[0].grid(True, linestyle='--', alpha=0.3)

        # Loss plot
        ax[1].plot(range(epoch+1), self.loss_history[:epoch+1], 'b-')
        ax[1].set_title('Training Loss (Categorical Cross-Entropy)')
        ax[1].set_xlabel('Epoch')
        ax[1].set_ylabel('Loss')
        ax[1].grid(True, linestyle='--', alpha=0.3)

        fig.canvas.draw()
        image = np.array(fig.canvas.renderer.buffer_rgba())
        self.frames.append(image)
        plt.close(fig)

# Generate multiclass data
X, y = make_classification(n_samples=200, n_features=2, n_classes=3, 
                           n_clusters_per_class=1, n_redundant=0, random_state=42)

# Train the model
model = LogisticRegression(learning_rate=0.5, n_epochs=100)
model.fit(X, y, save_path="Multiclass_LogisticRegression.gif")

print("\n=== Final Model ===")
print(f"Weights:\n{model.weights}")
print(f"Bias:\n{model.bias}")
print(f"Final Cross-Entropy Loss: {model.loss_history[-1]:.4f}")



=== Final Model ===
Weights:
[[ 0.34170275  2.14025     0.51804724]
 [ 2.00078956  1.68144826 -0.68223782]]
Bias:
[10.07882439 10.32929956  9.59187605]
Final Cross-Entropy Loss: 0.5109
